# Großwetterlagen Analysis

This notebook aims to analyze Großwetterlagen (GWL). 

For each day where a specific GWL appeared, a plot is created for the $500 hPa$ pressure level that shows:

- geopotential heights
- temperature
- wind speed and direction
- temperature, geopotential height and wind speed/direction at each individual appearance
This allows to see what similarities these quantities share for each day where the specific GWL appeared.
We further want to analyze GWL statistics:

- abundance
- stability (duration mean, std, etc.)
- forecast uncertainty per GWL


In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import a6
import typing as t
import xarray as xr
import math
import matplotlib.pyplot as plt
import numpy as np
import pathlib
import cartopy.crs as ccrs

In [ ]:
path = pathlib.Path("/home/fabian/Documents/MAELSTROM/gwl")

gwl = xr.open_dataset("../data/gwl.nc")
modes = a6.modes.methods.determine_lifetimes_of_modes(gwl["GWL"])
scores = xr.open_dataset("../data/scores.nc")

data = xr.open_dataset(
    "/home/fabian/Documents/MAELSTROM/data/pca/pressure_level_500_950_daily_mean_2017_2020.nc"
)

In [ ]:
data = (
    a6.datasets.methods.select.select_levels(levels=500)
    >> a6.features.methods.geopotential.calculate_geopotential_height()
).apply_to(data)

In [ ]:
for mode in modes:
    datetimes = list(
        mode.get_dates(start=data["time"][0], end=data["time"][-1])
    )
    fig, _ = a6.plotting.plot_combined(
        data=data,
        dates=datetimes,
    )
    fig.savefig(path / f"gwl_{mode.label}.png")

In [ ]:
fig, _ = a6.plotting.plot_modes_durations(modes)
fig.savefig(path / "gwls.pdf")

In [ ]:
scores_per_mode = a6.evaluation.modes.evaluate_scores_per_mode(
    modes, scores=[scores]
)
scores_per_mode